# AetherMindAI Launcher 🚀

Welcome to the AetherMindAI Launcher notebook! This notebook will guide you through cloning the repository, installing dependencies, and launching the AetherMindAI application in either a terminal chat loop or a Gradio-based web interface.

## 1. Clone AetherMindAI Repository

This cell clones the AetherMindAI GitHub repository into your Colab environment and navigates into the project directory.

In [ ]:
import os
try:
    repo_url = "https://github.com/Divpw/AetherMindAI.git"
    repo_name = "AetherMindAI"

    if os.path.isdir(repo_name):
        print(f"Repository '{repo_name}' already cloned. Skipping clone.")
    else:
        print(f"Cloning repository from {repo_url}...")
        !git clone {repo_url}
        print("Repository cloned successfully!")

    # Change directory into the repository
    os.chdir(repo_name)
    print(f"Changed current directory to: {os.getcwd()}")

    print("\nRepository contents:")
    !ls -lA

    print("\n✅ GitHub Integration cell completed successfully.")

except Exception as e:
    print(f"❌ An error occurred during GitHub Integration: {e}")

## 2. Install Dependencies

This cell installs all the necessary Python libraries listed in the `requirements.txt` file. We use the `--upgrade` flag to ensure the latest compatible versions are installed.

In [ ]:
import os
try:
    # Ensure we are in the repository directory
    repo_name = "AetherMindAI"
    if not os.getcwd().endswith(repo_name):
        # This might happen if the notebook is run out of order or restarted after cloning
        if os.path.isdir(repo_name):
            os.chdir(repo_name)
            print(f"Changed directory to {os.getcwd()} to install dependencies.")
        else:
            raise FileNotFoundError(f"Repository directory '{repo_name}' not found. Please run the clone step first.")
    
    requirements_file = "requirements.txt"
    if not os.path.exists(requirements_file):
        print(f"❌ Error: '{requirements_file}' not found in {os.getcwd()}.")
        print("Please ensure the repository was cloned correctly and you are in the repo directory.")
    else:
        print(f"Installing dependencies from {requirements_file}...")
        !pip install -r {requirements_file} --upgrade
        print("\n✅ Dependencies installed successfully.")

except FileNotFoundError as fnf_error:
    print(f"❌ {fnf_error}")
except Exception as e:
    print(f"❌ An error occurred during dependency installation: {e}")

## 3. Check Environment

This cell performs checks on your Python environment, including the Python version, PyTorch installation, GPU availability, and the installed Gradio version.

In [ ]:
import sys
import importlib

print("--- Environment Checks ---")
try:
    # 1. Python Version
    print(f"\n🐍 Python Version:")
    !python --version

    # 2. PyTorch and GPU Status
    print(f"\n🔥 PyTorch & GPU Status:")
    try:
        import torch
        print(f"  PyTorch Version: {torch.__version__}")
        cuda_available = torch.cuda.is_available()
        print(f"  CUDA Available: {cuda_available}")
        if cuda_available:
            print(f"  GPU Name: {torch.cuda.get_device_name(0)}")
            print(f"  Current CUDA Device: {torch.cuda.current_device()}")
            print(f"  Torch Device: cuda")
        else:
            print("  No GPU detected by PyTorch. Operations will use CPU.")
            print(f"  Torch Device: cpu")
    except ImportError:
        print("  ❌ PyTorch is not installed. Please check the dependency installation step.")
    except Exception as e:
        print(f"  ❌ Error during PyTorch check: {e}")

    # 3. Gradio Version
    print(f"\n🎨 Gradio Version:")
    try:
        # Using importlib to check version as !pip show might be slow or verbose
        gradio_spec = importlib.util.find_spec("gradio")
        if gradio_spec:
            import gradio
            print(f"  Gradio Version: {gradio.__version__}")
        else:
            print("  ❌ Gradio is not installed. Please check the dependency installation step.")
    except ImportError:
         print("  ❌ Gradio is not installed or importable. Please check the dependency installation step.")
    except Exception as e:
        print(f"  ❌ Error during Gradio check: {e}")
        print("  Attempting !pip show gradio as fallback...")
        !pip show gradio | grep Version

    print("\n✅ Environment checks completed.")

except Exception as e:
    print(f"❌ An error occurred during environment checks: {e}")

## 4. Load AI Modules & Set Up Core Components

This cell dynamically loads the core Python modules of AetherMindAI. It initializes the language model, tokenizer, and other tools. Key components like the model, tokenizer, and processing functions are loaded once here to be used by the chosen interface. Flags are set to indicate availability, bypassing fallback logic in the modules.

In [ ]:
import sys
import os
import importlib.util

print("--- Loading AI Modules & Setting Up Core Components ---")

# Ensure we are in the 'AetherMindAI' directory where 'src' is located
expected_dir_name = "AetherMindAI"
if os.path.basename(os.getcwd()) != expected_dir_name:
    if os.path.isdir(expected_dir_name):
        os.chdir(expected_dir_name)
        print(f"Changed directory to: {os.getcwd()}")
    else:
        # If 'AetherMindAI' directory doesn't exist at all in current path, error out.
        # This case should ideally be caught by the clone step, but as a safeguard:
        print(f"❌ Error: Base directory '{expected_dir_name}' not found. Please run the clone step first.")
        # sys.exit() # Or raise an exception if preferred in a notebook context

# Add src to Python path
src_path = os.path.join(os.getcwd(), "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)
print(f"Added '{src_path}' to sys.path.")

# --- Global variables for loaded components ---
notebook_model = None
notebook_tokenizer = None
notebook_generate_response = None
notebook_solve_math_query = None
notebook_run_ml_task = None
notebook_run_reasoning_prompt = None
notebook_aethermind_chat_loop = None # Will be from aethermind_main
notebook_launch_gradio_interface = None

phi2_available_flag = False
math_solver_available_flag = False
ml_tools_available_flag = False
reasoning_engine_available_flag = False
aethermind_main_module_loaded_flag = False
gradio_app_module_loaded_flag = False

try:
    # 1. Load phi2_colab_runner
    print("\n1. Loading Phi-2 Model and Tokenizer...")
    try:
        import phi2_colab_runner
        notebook_model, notebook_tokenizer = phi2_colab_runner.load_model_and_tokenizer()
        if notebook_model and notebook_tokenizer:
            notebook_generate_response = phi2_colab_runner.generate_response
            phi2_available_flag = True
            print("   ✅ Phi-2 model, tokenizer, and response function loaded successfully.")
        else:
            print("   ❌ Failed to load Phi-2 model or tokenizer. Chat functionality will be impaired.")
    except ImportError:
        print("   ❌ Error: Failed to import 'phi2_colab_runner'. Chat functionality will be unavailable.")
    except Exception as e:
        print(f"   ❌ An unexpected error occurred during Phi-2 initialization: {e}")

    # 2. Load colab_math_solver
    print("\n2. Loading Math Solver...")
    try:
        import colab_math_solver
        notebook_solve_math_query = colab_math_solver.solve_math_query
        math_solver_available_flag = True
        print("   ✅ Math solver loaded successfully.")
    except ImportError:
        print("   ❌ Error: Failed to import 'colab_math_solver'. Math solving functionality will be unavailable.")
    except Exception as e:
        print(f"   ❌ An unexpected error occurred loading math_solver: {e}")

    # 3. Load colab_ml_tools
    print("\n3. Loading ML Tools...")
    try:
        import colab_ml_tools
        notebook_run_ml_task = colab_ml_tools.run_ml_task
        ml_tools_available_flag = True
        print("   ✅ ML tools loaded successfully.")
    except ImportError:
        print("   ❌ Error: Failed to import 'colab_ml_tools'. ML task functionality will be unavailable.")
    except Exception as e:
        print(f"   ❌ An unexpected error occurred loading ml_tools: {e}")

    # 4. Load reasoning_engine
    print("\n4. Loading Reasoning Engine...")
    try:
        import reasoning_engine
        notebook_run_reasoning_prompt = reasoning_engine.run_reasoning_prompt
        reasoning_engine_available_flag = True # Added this flag for consistency
        print("   ✅ Reasoning engine loaded successfully.")
    except ImportError:
        print("   ❌ Error: Failed to import 'reasoning_engine'. Reasoning functionality will be unavailable.")
    except Exception as e:
        print(f"   ❌ An unexpected error occurred loading reasoning_engine: {e}")

    # 5. Load aethermind_main (for chat loop and utility functions)
    # We will inject our loaded components into its namespace.
    print("\n5. Preparing AetherMind Main Controller...")
    aethermind_main_module = None
    try:
        spec_main = importlib.util.find_spec("aethermind_main")
        if spec_main:
            aethermind_main_module = importlib.util.module_from_spec(spec_main)
            sys.modules['aethermind_main'] = aethermind_main_module # Add to sys.modules before exec
            spec_main.loader.exec_module(aethermind_main_module)

            # Inject notebook-loaded components and set flags
            aethermind_main_module.phi2_model = notebook_model
            aethermind_main_module.phi2_tokenizer = notebook_tokenizer
            aethermind_main_module.generate_response = notebook_generate_response # If it uses this name
            aethermind_main_module.phi2_available = phi2_available_flag

            aethermind_main_module.solve_math_query = notebook_solve_math_query
            aethermind_main_module.math_solver_available = math_solver_available_flag

            aethermind_main_module.run_ml_task = notebook_run_ml_task
            aethermind_main_module.ml_tools_available = ml_tools_available_flag
            
            # If reasoning_engine functions are used by aethermind_main, inject them too.
            # Assuming aethermind_main doesn't directly use reasoning_engine for its core loop.

            notebook_aethermind_chat_loop = aethermind_main_module.aethermind_chat_loop
            aethermind_main_module_loaded_flag = True
            print("   ✅ AetherMind main controller prepared and components injected.")
        else:
            print("   ❌ Error: aethermind_main.py spec not found.")
    except Exception as e:
        print(f"   ❌ An error occurred preparing aethermind_main: {e}")

    # 6. Load aethermind_gradio_app (for Gradio interface)
    # We will inject our loaded components into its namespace.
    print("\n6. Preparing AetherMind Gradio App...")
    aethermind_gradio_app_module = None
    try:
        spec_gradio = importlib.util.find_spec("aethermind_gradio_app")
        if spec_gradio:
            aethermind_gradio_app_module = importlib.util.module_from_spec(spec_gradio)
            sys.modules['aethermind_gradio_app'] = aethermind_gradio_app_module # Add to sys.modules
            spec_gradio.loader.exec_module(aethermind_gradio_app_module)

            # Inject notebook-loaded components into Gradio app module
            aethermind_gradio_app_module.CHAT_MODEL = notebook_model
            aethermind_gradio_app_module.CHAT_TOKENIZER = notebook_tokenizer
            aethermind_gradio_app_module.CHAT_MODEL_LOADED = phi2_available_flag
            # The Gradio app defines its own wrapper, but relies on CHAT_MODEL_LOADED and the globals.
            # It also tries to import other tools directly. We need to override those too.
            aethermind_gradio_app_module.solve_math_query = notebook_solve_math_query
            aethermind_gradio_app_module.run_ml_task = notebook_run_ml_task
            aethermind_gradio_app_module.run_reasoning_prompt = notebook_run_reasoning_prompt

            notebook_launch_gradio_interface = aethermind_gradio_app_module.launch_gradio_interface
            gradio_app_module_loaded_flag = True
            print("   ✅ AetherMind Gradio app prepared and components injected.")
        else:
            print("   ❌ Error: aethermind_gradio_app.py spec not found.")
    except Exception as e:
        print(f"   ❌ An error occurred preparing aethermind_gradio_app: {e}")

    print("\n✅ AI Modules loading and setup process completed.")
    print("--- Summary of Loaded Components ---")
    print(f"Phi-2 Model & Tokenizer: {'Available' if phi2_available_flag else 'Unavailable'}")
    print(f"Math Solver: {'Available' if math_solver_available_flag else 'Unavailable'}")
    print(f"ML Tools: {'Available' if ml_tools_available_flag else 'Unavailable'}")
    print(f"Reasoning Engine: {'Available' if reasoning_engine_available_flag else 'Unavailable'}")
    print(f"AetherMind Main Controller: {'Prepared' if aethermind_main_module_loaded_flag else 'Not Prepared'}")
    print(f"AetherMind Gradio App: {'Prepared' if gradio_app_module_loaded_flag else 'Not Prepared'}")

except Exception as e:
    print(f"❌ A major error occurred during the module loading and setup process: {e}")



## 5. Select Interface and Launch AetherMindAI

Use the dropdown menu below to select how you want to interact with AetherMindAI, then run this cell to launch the application.

- **Terminal Chat Loop**: A text-based chat interface running directly in the notebook output cell.
- **Gradio Interface**: A richer, web-based graphical user interface.

In [ ]:
#@title Select Interface and Launch AetherMindAI
interface_choice = "Terminal Chat Loop" #@param ["Terminal Chat Loop", "Gradio Interface"]

print("--- Launching AetherMindAI Interface ---")
try:
    if interface_choice == "Terminal Chat Loop":
        print("Selected: Terminal Chat Loop. Initializing...")
        if aethermind_main_module_loaded_flag and callable(notebook_aethermind_chat_loop):
            # Ensure the main module is using the correct, notebook-loaded model and tokenizer
            if hasattr(aethermind_main_module, 'phi2_model') and hasattr(aethermind_main_module, 'phi2_tokenizer'):
                aethermind_main_module.phi2_model = notebook_model
                aethermind_main_module.phi2_tokenizer = notebook_tokenizer
                aethermind_main_module.generate_response = notebook_generate_response 
                aethermind_main_module.phi2_available = phi2_available_flag
                
                aethermind_main_module.solve_math_query = notebook_solve_math_query
                aethermind_main_module.math_solver_available = math_solver_available_flag
                
                aethermind_main_module.run_ml_task = notebook_run_ml_task
                aethermind_main_module.ml_tools_available = ml_tools_available_flag
                
                print("Launching Terminal Chat Loop...")
                notebook_aethermind_chat_loop() # Call the function from the prepared module
            else:
                print("❌ Error: aethermind_main module is missing expected model/tokenizer attributes.")
        else:
            print("❌ Error: Terminal chat loop function is not available. Please check the module loading step.")
            if not phi2_available_flag:
                print("   Reason: Core Phi-2 model/tokenizer failed to load.")

    elif interface_choice == "Gradio Interface":
        print("Selected: Gradio Interface. Initializing...")
        if gradio_app_module_loaded_flag and callable(notebook_launch_gradio_interface):
            # Ensure the Gradio app module uses the correct, notebook-loaded components
            if hasattr(aethermind_gradio_app_module, 'CHAT_MODEL') and \
               hasattr(aethermind_gradio_app_module, 'CHAT_TOKENIZER'):
                aethermind_gradio_app_module.CHAT_MODEL = notebook_model
                aethermind_gradio_app_module.CHAT_TOKENIZER = notebook_tokenizer
                aethermind_gradio_app_module.CHAT_MODEL_LOADED = phi2_available_flag
                # Also re-assign the tool functions it might try to import internally
                aethermind_gradio_app_module.solve_math_query = notebook_solve_math_query
                aethermind_gradio_app_module.run_ml_task = notebook_run_ml_task
                aethermind_gradio_app_module.run_reasoning_prompt = notebook_run_reasoning_prompt
                
                print("Launching Gradio Interface... (This may take a moment to start up)")
                notebook_launch_gradio_interface() # Call the function from the prepared module
            else:
                print("❌ Error: aethermind_gradio_app module is missing expected model/tokenizer attributes.")
        else:
            print("❌ Error: Gradio interface function is not available. Please check the module loading step.")
            if not phi2_available_flag:
                 print("   Reason: Core Phi-2 model/tokenizer failed to load, which Gradio UI depends on.")
    else:
        print(f"❌ Error: Unknown interface choice '{interface_choice}'.")

except NameError as ne:
    print(f"❌ NameError: A required variable or module was not defined: {ne}")
    print("   This usually means a previous cell (like Module Loader) did not run successfully or was skipped.")
except Exception as e:
    print(f"❌ An unexpected error occurred while launching the interface: {e}")



--- Launching AetherMindAI Interface ---
